<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pydrive

In [1]:
# ! pip install geemap

In [2]:
import sys
import os
import geemap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [20]:
def get_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()

    collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filter(ee.Filter.calendarRange(6, 10, 'month'))
      .filterDate('2016-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
    top = collection.limit(1)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
#     print(roi)
    img = top.first().clip(roi)
    return img#, geoid
    

def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

def init_dir(subdir, fn):
  if os.path.exists(subdir):
      if fn in os.listdir(subdir):
#         print('pass')
        return True
     
  if not os.path.exists(subdir):
      print('creating directory')
      createDir(subdir)
      return True

In [6]:
import ee
# ee.Authenticate()
ee.Initialize()

In [21]:
tract_data_dir = r'../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'
temp_tract_data_dir = "../Tracts/nyc_metro_boundaries/temp.shp"
temp_envelope_data_dir = "../Tracts/nyc_metro_boundaries/temp_envelope.shp"

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 
# shapefile['geometry'] = shapefile['geometry'].envelope

In [23]:
fn = 'least_cloudy_m6_m10_rectangle_highres.tif'

In [ ]:
# clip or not clip the images to census polygon
earth_engine_clip_image_to_geometry = False

In [17]:
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(top, visParams, 'tracts')
# Map

# Run to download Landsat Images

In [31]:
save_dir = 'download_landsat_images'
count = 0
for interval in range(0, shapefile.shape[0], 20): 
    try:
        sub = shapefile.iloc[interval:interval+20]
    except:
        sub = shapefile.iloc[interval:-1]
        
    # Convert tract polygon to the envelope polygon
    if earth_engine_clip_image_to_geometry == False:
        sub['geometry'] = sub['geometry'].envelope
        
    sub.to_file(temp_envelope_data_dir)
    tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_img)
    
    # download loop
    for i in range(0, sub.shape[0]):
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        filename = f"{subdir}/" + fn
        
        if os.path.exists(filename):
            continue
            
        init_dir(subdir, filename)
                 
        img = ee.Image(result.get(i))
        img = img.select(['B2', 'B3', 'B4', 'B5'])
                
        try:
            if img.projection().nominalScale().getInfo() != 30:
                count +=1
                continue
        except:
            print('--->Error getting projection. Passing...')
        
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False
            )
        print('---')

#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7', 'B8'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4', 'B5'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )
            
#         if not os.path.exists(filename):
#             img = img.select(['B1', 'B2', 'B3','B4'])
#             geemap.ee_export_image(
#                     img, filename=filename, file_per_band=False
#                 )

        
#         if os.path.exists(filename): 
#             print('saved')
      

    print(f'-----DONE {interval} - {interval + sub.shape[0]}')
print(f'Number of images with scale not at 30 m resolution: {count}')
    

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009350400/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
An error occurred while downloading.
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352701/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352702/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352800/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-grap

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352200/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352500/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352600/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09009352100/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/09005320200/least_cloud

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


...

Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34037373700/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34037373800/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34037373400/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039034500/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34039034600/least_cloud

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017017200/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017002300/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017001000/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017015900/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34017013800/least_cloud

/home/julia/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035054302/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035054301/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035052604/least_cloudy_m6_m10_rectangle_highres.tif
---
Generating URL ...
Please wait ...
Data downloaded to /mnt/e/julia/regional-representations-graph-model/dataset/earth_engine/download_landsat_images/34035052605/least_cloudy_m6_m10_rectangle_highres.tif
---
-----DONE 5500 - 5504
Number of images with scale not at 30 m resolution: 0


In [62]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(top, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
# Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

# Run for rectangle census tract images

In [34]:
def get_regions_to_retry():
    l = []
    counter=0
    for root, dirs, files in os.walk("download_landsat_images/"):
        if '.ipynb_checkpoints' in root:
            continue
        if fn not in files:
            l.append(root)
            continue
        else:
            counter+=1
        
            
    l.remove('download_landsat_images/')
    al = len(os.listdir("download_landsat_images/"))
    print(f'Num folders total: {al}')
    print(f'Num target files total: {counter}')
    print()
    l = [item[-11:] for item in l]
    return l
    
r = get_regions_to_retry()
print(len(r))
# for geoid in [r]:
#     shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
#     tracts = geemap.shp_to_ee(temp_tract_data_dir)
    
#     # Convert tract polygon to the envelope polygon
#     envelope_shp = shapefile.loc[shapefile.GEOID == geoid]
#     envelope_shp['geometry'] = envelope_shp['geometry'].envelope
#     envelope_shp.to_file(temp_envelope_data_dir)
#     envelope_tracts = geemap.shp_to_ee(temp_envelope_data_dir)
    
#     img = get_img(geoid, envelope_tracts)
#     save_dir = 'download_landsat_images'
#     subdir = f"{save_dir}/{geoid}"
#     filename = f"{subdir}/" + fn

# #     img = ee.Image(result.get(i))
#     img = img.select(['B2', 'B3', 'B4', 'B5'])

#     try:
#         if img.projection().nominalScale().getInfo() != 30:
#             count +=1
#             continue
#     except:
#         print('--->Error getting projection. Passing...')

#     geemap.ee_export_image(
#             img, filename=filename, file_per_band=False)


#     print('--\n')

Num folders total: 8232
Num target files total: 5503

2728


In [39]:
df.shape[0]

5504

In [ ]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map

# Test package



In [22]:
cld

In [23]:
img

In [27]:
# my scenario
ee.Initialize()

# aoi = ee.Geometry(Point([-48.4, -23]))
# aoi = shapefile.iloc[0][1]
# img = ee.Image(result.get(i))
        
        
# clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)


Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B2'], 'min': 0, 'max': 0.2}
Map.addLayer(img, visParams, 'band 2')
Map


# Map = geemap.Map(center=(-23, -48.4), zoom=9)
# Map.addLayer(img, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
# Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [21]:
ee.Initialize()

aoi = ee.Geometry(Point([-48.4, -23]))

clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate('2019-11-25', '2019-11-28').filterBounds(aoi)

# img = s2.first()
cld = clouds.first()

Map = geemap.Map(center=(-23, -48.4), zoom=9)
Map.addLayer(cld, {'min': 0, 'max': 100, 'palette': ['purple', 'yellow']}, 'Cloud Probability', True, 1)
Map

Map(center=[-23, -48.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Scratchwork

In [ ]:
import tqdm


stats = pd.DataFrame(columns = [
    'geoid',
    'max',
 'mean',
 'min',
 'sample_sd',
 'sample_var',
 'sum',
 'sum_sq',
 'total_count',
 'total_sd',
 'total_var',
 'valid_count',
 'weight_sum',
 'weighted_sum'])

img_list = ee.List([])


def save_images(geoid):
    print(geoid_list)
    for geoid in geoid_list:
        print(geoid)
    save_dir = 'download_landsat_images/'
    subdir = f"{save_dir}{geoid}"


    if os.path.exists(subdir):
        if len(os.listdir(subdir)) >= 10:
#               print('pass')
          return True
        else:
#               print(f'Redoing: {subdir}')
            pass

    if not os.path.exists(subdir):
#             print('creating directory')
        createDir(subdir)

    
    geemap.ee_export_image(img, filename=f'{subdir}/least_CC.tif', region=roi, scale=90)

    # for image collection stats 
#         temp = pd.DataFrame(data=img.toArray().getInfo()).T
#         temp.columns=temp.keys().getInfo()
#         temp['geoid'] = geoid

#         stats = pd.concat([stats, temp], axis=0)
#         stats.to_csv('save_dir/_CC_stats.csv', index=False)


#         for img_idx in top.aggregate_array('system:index').getInfo():
#             img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
#             filename = f"{subdir}/{img_idx}.tif"

#             new_img = img.clip(roi)

#             geemap.ee_export_image(
#                 new_img, filename=filename, scale=90, region=roi, file_per_band=False
#             )

    return 

In [ ]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [ ]:
save_dir = 'download_landsat_images/'
def Count_files_in_subd():
    l = []
    
    for root, dirs, files in os.walk(save_dir):
        # print("{} in {}".format(len(files), root))
        l.append(len(files))
#         if len(files) == 0:
#           print(dirs)
    al = len(os.listdir(save_dir))
    print(f'Num folders total: {al}')
    print(f'Num folders with 0 files: {l.count(0)}')
    print(f'Num folders with >0 files: {al - l.count(0)}')
    print()
Count_files_in_subd()